In [37]:
import pandas as pd
import sqlite3
from sqlite3 import Error
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

try:
    conn = sqlite3.connect("./database.sqlite")
except Error as e:
    print(e)


    
reddit_comments = conn.cursor().execute("SELECT subreddit_id, name, body  from May2015 LIMIT 5").fetchall()
reddit_comments[0]

('t5_378oi', 't1_cqug90g', 'くそ\n読みたいが買ったら負けな気がする\n図書館に出ねーかな')

In [38]:
"""
Now that I have data in tuple format, how do i clean up the data?
Next step is to tokenize -> transform comments to an actual value of some sort.
"""
sid = SentimentIntensityAnalyzer()
for comment in reddit_comments:
    print(comment)
    array_of_sentences = nltk.sent_tokenize(comment[2])
    for sentence in array_of_sentences:
        ss = sid.polarity_scores(sentence)
        array_of_tokenized_values = []
        for k in sorted(ss):
            print('{0}: {1}, '.format(k, ss[k]), end='')
            print()

('t5_378oi', 't1_cqug90g', 'くそ\n読みたいが買ったら負けな気がする\n図書館に出ねーかな')
['くそ\n読みたいが買ったら負けな気がする\n図書館に出ねーかな']
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
('t5_2qo4s', 't1_cqug90h', "gg this one's over. off to watch the NFL draft I guess")
["gg this one's over.", 'off to watch the NFL draft I guess']
compound: 0.296, 
neg: 0.0, 
neu: 0.577, 
pos: 0.423, 
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
('t5_2cneq', 't1_cqug90i', "Are you really implying we return to those times or anywhere near that political environment?  If so, you won't have much luck selling the American people on that governance concept without ushering in American Revolution 2.0.")
['Are you really implying we return to those times or anywhere near that political environment?', "If so, you won't have much luck selling the American people on that governance concept without ushering in American Revolution 2.0."]
compound: 0.0, 
neg: 0.0, 
neu: 1.0, 
pos: 0.0, 
compound: -0.357, 
neg: 0.11, 
neu: 0.89, 
pos: 0.0, 
('t5_2qh1

In [ ]:
"""
Alright now have to perform sentiment analysis
"""